In [ ]:
import pandas as pd
import pyodbc

# Connecting to SQL Server
driver = "{SQL Server}"  # Adjust based on your configuration
server = "DESKTOP-UFKKVAB"
database = "Ideas en Verde Historical Records"

conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;')
cursor = conn.cursor()

# Loading CSV into DataFrame
csv_file = "tabla_larga_corregida_manual_2024.csv"
df = pd.read_csv(csv_file)

# Checking data
df.info()
print(df.head())

# Converting date column
df['Date'] = pd.to_datetime(df['Date'])

# Transforming data to long format
df_long = df.melt(id_vars=['Date'], var_name='Métrica', value_name='Valor')

# Extracting client number
df_long['Cliente'] = df_long['Métrica'].str.extract(r'_(\d+)')
df_long['Cliente'] = 'Cliente ' + df_long['Cliente'].astype(str)

# Replacing underscore with slash in metric names
df_long['Métrica'] = df_long['Métrica'].str.replace('_', '/')

# Creating new table in SQL Server
cursor.execute("""
    CREATE TABLE tabla_larga_cliente_2024 (
        Date DATETIME,
        Métrica VARCHAR(50),
        Valor FLOAT,
        Cliente VARCHAR(50),
        NumericPart INT
    )
""")
conn.commit()

# Inserting data into SQL Server
insert_query = "INSERT INTO tabla_larga_cliente_2024 (Date, Métrica, Valor, Cliente, NumericPart) VALUES (?, ?, ?, ?, ?)"
for index, row in df_long.iterrows():
    cursor.execute(insert_query, row['Date'], row['Métrica'], row['Valor'], row['Cliente'], int(row['Cliente'].split()[1]))
conn.commit()

# Closing connection
cursor.close()
conn.close()

print("Data successfully loaded into SQL Server and transformed.")
